In [1]:
import networkx as nx
neighbors_fpath = "model/text8.graph"
G = nx.read_edgelist(neighbors_fpath, nodetype=str, delimiter="\t", data=(('weight',float),))

[(0, {'weight': 999}),
 (1, {'weight': 999}),
 (2, {'weight': 999}),
 (3, {'weight': 999}),
 (4, {'weight': 999}),
 (5, {'weight': 999}),
 (6, {'weight': 999}),
 (7, {'weight': 999}),
 (8, {'weight': 999}),
 (9, {'weight': 999}),
 (10, {'weight': 999}),
 (11, {'weight': 999}),
 (12, {'weight': 999}),
 (13, {'weight': 999}),
 (14, {'weight': 999}),
 (15, {'weight': 999}),
 (16, {'weight': 999}),
 (17, {'weight': 999}),
 (18, {'weight': 999}),
 (19, {'weight': 999})]

In [38]:
x = G["the"]
G["-----------"].keys()

KeyError: '-----------'

In [43]:
import codecs 
from multiprocessing import Pool
from chinese_whispers import chinese_whispers, aggregate_clusters
import networkx as nx 
from time import time


def get_ego_network_simple(ego):
    res = "{}\t{}\t{}\n".format(ego, len(G.nodes), len(G[ego]))
    return res


def get_ego_network(ego):
    tic = time()
    ego_network = nx.Graph(name=ego)

    # Add related and substring nodes
    substring_nodes = []
    for j, node in enumerate(G.nodes):
        if ego.lower() == node.lower():
            ego_nodes = [(rn, {"weight": G[node][rn]["weight"]}) for rn in G[node].keys()]
            ego_network.add_nodes_from(ego_nodes)
        else:
            if "_" not in node: continue
            if node.startswith(ego + "_") or node.endswith("_" + ego):

                if ego in G and node in G[ego]: w = G[ego][node]["weight"]
                else: w = 0.99

                substring_nodes.append( (node, {"weight": w}) )
    ego_network.add_nodes_from(substring_nodes)

    # Find edges of the ego network
    for r_node in ego_network:
        related_related_nodes = G[r_node]
        related_related_nodes_ego = sorted(
            [(related_related_nodes[rr_node]["weight"], rr_node) for rr_node in related_related_nodes if rr_node in ego_network],
            reverse=True)[:n]
        related_edges = [(r_node, rr_node, {"weight": w}) for w, rr_node in  related_related_nodes_ego]
        ego_network.add_edges_from(related_edges)

    chinese_whispers(ego_network, weighting="top", iterations=20)
    if verbose: print("{}\t{:f} sec.".format(ego, time()-tic))
    
    output_lines = []
    for label, cluster in sorted(aggregate_clusters(ego_network).items(), key=lambda e: len(e[1]), reverse=True):
        output_lines.append("{}\t{}\t{}\t".format(
            ego_network.name,
            len(output_lines)+1,
            ", ".join( ["{}:{:.4f}".format(c_node, ego_network.node[c_node]["weight"]) for c_node in cluster] )
        ))
    
    return "\n".join(output_lines)


def ego_network_clustering(neighbors_fpath, clusters_fpath, max_related=300, num_cores=32):
    # load the server here: the G is needed 
    # also pass the n variable in the right way
    
    with codecs.open(clusters_fpath, "w", "utf-8") as output, Pool(num_cores) as pool:
        output.write("word\tcid\tcluster\tisas\n")

        for i, ego_network in enumerate(pool.imap_unordered(get_ego_network, G.nodes)):
            if i % 100 == 0: print(i, "ego networks processed")
            sense_num = 1
            for label, cluster in sorted(aggregate_clusters(ego_network).items(), key=lambda e: len(e[1]), reverse=True):
                output.write("{}\t{}\t{}\t\n".format(
                    ego_network.name,
                    sense_num,
                    ", ".join( ["{}:{:.4f}".format(c_node, ego_network.node[c_node]["weight"]) for c_node in cluster] )
                ))
                sense_num += 1
    print("Clusters:", clusters_fpath)
    


414
376
418
428
417
415
389
401
423
386
367
416
427
401
373
421
409
409
402
416
385
394
394
382
379
368
371
366
365
433
359
385
355
391
352
346
357
it	0.097710 sec.
355
in	0.104901 sec.
are	0.104028 sec.
autism	0.107267 sec.
autistic	0.106240 sec.
the	0.107776 sec.
have	0.106283 sec.
356
348
319
343
332
355
392
386
about	0.122986 sec.
with	0.127928 sec.
325
that	0.129672 sec.
340
to	0.131024 sec.
on	0.030844 sec.
347
336
these	0.123048 sec.
384
319
317
from	0.128600 sec.
s	0.130283 sec.
or	0.144974 sec.
338
335
one	0.035637 sec.
some	0.146500 sec.
increase	0.036607 sec.
323
albedo	0.147778 sec.
as	0.147487 sec.
344
disorder	0.138841 sec.
an	0.141369 sec.
of	0.151823 sec.
317
asperger_s	0.138174 sec.
328
for	0.151584 sec.
329
320
278
304
329
other	0.047316 sec.
343
a	0.157575 sec.
and	0.159420 sec.
316
311
294
335
they	0.151598 sec.
can	0.054103 sec.
anarchists	0.053208 sec.
social	0.054053 sec.
this	0.165946 sec.
284
311
317
315
their	0.161511 sec.
325
also	0.048891 sec.
is	0.183732 se

winning_authors	0.048645 sec.
cum_laude	0.053566 sec.
ulysses_s	0.039841 sec.
blockbusters	0.041328 sec.
223
223
230
214
266
224
232
231
252
240
218
210
230
consistently	0.033612 sec.
237
234
219
specialty	0.072450 sec.
severely_limited	0.047510 sec.
while	0.060143 sec.
diligent	0.060525 sec.
serrano	0.048074 sec.
ganglia	0.031811 sec.
sociology	0.050989 sec.
217
212
cowdery	0.050353 sec.
224
mips_computer	0.050243 sec.
216
218
rapid_fire	0.042540 sec.
211
209
mount_everest	0.043878 sec.
203
218
205
play	0.049660 sec.
area	0.038692 sec.
233
203
200
206
225
parents	0.044228 sec.
hobbies	0.045927 sec.
uk	0.049090 sec.
201
commenting_on	0.052692 sec.
222
213
leading_figure	0.044654 sec.
200 ego networks processed
300 ego networks processed
malts	0.076060 sec.
classmate	0.058579 sec.
human	0.050871 sec.
usually	0.041849 sec.
203
gia_carangi	0.047914 sec.
203
np	0.045311 sec.
ordinal_numbers	0.063632 sec.
204
206
disjoint_union	0.048362 sec.
203
e_g	0.052389 sec.
cell_walls	0.051917 sec.
ex

insufficiently	0.080233 sec.
hulk_hogan	0.053524 sec.
proposed	0.049586 sec.
inventory	0.049797 sec.
longevity	0.050857 sec.
valign_top	0.061053 sec.
240
201
204
feast_day	0.038247 sec.
207
214
202
mired	0.047961 sec.
sprengel	0.034596 sec.
208
219
currency_exchange	0.049510 sec.
218
re_recording	0.034785 sec.
rebel_against	0.046394 sec.
218
concave	0.049106 sec.
205
enable	0.065750 sec.
218
201
208
idealism	0.084974 sec.
213
criminal_acts	0.047615 sec.
occasional	0.049382 sec.
212
248
227
hippolyte	0.047824 sec.
217
buffalo	0.039202 sec.
nkrumah	0.049588 sec.
delhi_sultanate	0.042789 sec.
200
guinness_book	0.049497 sec.
202
twin_brother	0.054095 sec.
245
244
biting	0.047681 sec.
211
208
forth	0.026329 sec.
209
204
207
232
navies	0.041085 sec.
216
ansi	0.058721 sec.
conseil_g	0.041532 sec.
florida	0.043442 sec.
201
227
trailers	0.031801 sec.
204
withered	0.044750 sec.
203
opinions	0.040734 sec.
spires	0.027730 sec.
209
200
208
ingolstadt	0.073447 sec.
funchal	0.066948 sec.
221
219
ivan

212
205
napster	0.050246 sec.
disgrace	0.050507 sec.
201
221
213
goes_back	0.051255 sec.
atomic_clock	0.049228 sec.
death_rates	0.052044 sec.
investigative	0.049898 sec.
209
unreal_tournament	0.050011 sec.
209
computational_linguistics	0.050645 sec.
letting	0.049371 sec.
220
ballistic_missile	0.050543 sec.
215
200
219
deciduous_forests	0.053742 sec.
205
205
207
221
212
214
gillies	0.026873 sec.
208
daily_basis	0.047084 sec.
show_signs	0.046815 sec.
interim_president	0.049190 sec.
227
207
203
n	0.061010 sec.
excommunicated	0.047505 sec.
theodore_roosevelt	0.048675 sec.
203
203
203
204
interoperable	0.049057 sec.
bobby_fischer	0.066565 sec.
202
211
die	0.038879 sec.
226
245
condition	0.041069 sec.
212
regenerative	0.049510 sec.
222
significant_improvement	0.039792 sec.
205
208
research_indicates	0.057302 sec.
drinking_games	0.027910 sec.
234
209
millikan	0.038689 sec.
border_guard	0.048722 sec.
were_expelled	0.093260 sec.
koyaanisqatsi	0.046268 sec.
201
200
prehistoric_times	0.049249 sec

cbe	0.048842 sec.
krupp	0.047731 sec.
envisioned	0.048322 sec.
philadelphia_eagles	0.071358 sec.
216
infantryman	0.054459 sec.
230
218
208
blessed	0.048823 sec.
divorces	0.047960 sec.
software_license	0.064279 sec.
213
ice	0.048740 sec.
north_pacific	0.051615 sec.
221
200
211
208
205
210
addicted	0.064527 sec.
saint_kitts	0.052331 sec.
divisor	0.048503 sec.
health	0.049179 sec.
226
205
210
217
ecommerce	0.047251 sec.
206
polar	0.034635 sec.
concealment	0.048821 sec.
linspire	0.055197 sec.
209
202
207
doubly_linked	0.047888 sec.
228
laptop	0.050088 sec.
playfulness	0.050690 sec.
reformed	0.074851 sec.
washington_senators	0.049673 sec.
220
206
strictly_speaking	0.038461 sec.
210
248
ignition_system	0.049714 sec.
211
cofinality	0.049532 sec.
mehrgarh	0.050735 sec.
advocacy	0.048551 sec.
214
autumnal_equinox	0.069660 sec.
el_paso	0.047234 sec.
tudes	0.057582 sec.
lone	0.027015 sec.
229
216
222
207
energy_transfer	0.102895 sec.
double_precision	0.068352 sec.
still_survive	0.035425 sec.
nba	

...

208
202
205
205
width_align	0.045380 sec.
214
principal_cities	0.042757 sec.
zamenhof	0.032126 sec.
animal_farm	0.043743 sec.
ephesians	0.050881 sec.
201
liberalization	0.058220 sec.
205
202
224
ipa_symbols	0.045300 sec.
hydrogen_sulfide	0.037444 sec.
british_library	0.049367 sec.
204
potomac	0.052073 sec.
leads	0.051633 sec.
200
207
laplace_transform	0.051861 sec.
incessantly	0.031985 sec.
207
213
221
200
nag	0.038451 sec.
xxiv	0.037482 sec.
200
cdu	0.055408 sec.
tupper	0.038786 sec.
213
201
211
alan_alda	0.045523 sec.
204
202
200
200
200
burnt	0.045376 sec.
capoeira	0.054877 sec.
adept_at	0.053229 sec.
repelling	0.047898 sec.
underestimated	0.057209 sec.
varies	0.061650 sec.
202
handgun	0.050562 sec.
201
224
200
204
205
saving	0.058565 sec.
201
simmered	0.041142 sec.
202
203
before_joining	0.042321 sec.
historiographical	0.043869 sec.
talents	0.036312 sec.
headline	0.051767 sec.
nature_versus	0.056036 sec.
presbyter	0.030314 sec.
wadsworth	0.051069 sec.
korca	0.045586 sec.
201
67900 

deal_with	0.027727 sec.
mares	0.051812 sec.
203
204
206
200
201
limewire	0.055647 sec.
diarrhea_hepatitis	0.052779 sec.
204
athlon	0.031082 sec.
waltham	0.037207 sec.
destabilizing	0.030644 sec.
200
subfamilies	0.052825 sec.
avert	0.036788 sec.
208
202
kim	0.041636 sec.
200
caught_fire	0.051422 sec.
203
200
vista	0.043691 sec.
align	0.035737 sec.
information_about	0.053790 sec.
200
200
208
204
204
200
210
ellis_island	0.048072 sec.
alexandria	0.050542 sec.
205
polynesian	0.053615 sec.
201
wormholes	0.051535 sec.
posted	0.051261 sec.
banquo	0.029498 sec.
bravo	0.050873 sec.
209
203
reefs	0.048754 sec.
mid_nineteenth	0.054196 sec.
201
203
211
203
sandals	0.048882 sec.
210
farad_n	0.046111 sec.
mem	0.025982 sec.
marmoset_callithrix	0.033991 sec.
204
harpercollins_publishers	0.041047 sec.
amplifiers	0.042787 sec.
mason_remey	0.049642 sec.
200
218
smoked	0.046004 sec.
isolationist	0.029759 sec.
schematics	0.050036 sec.
211
205
ccd	0.031658 sec.
210
exploitation	0.053061 sec.
crannog	0.03385

202
max_weber	0.040111 sec.
210
exchequer	0.051856 sec.
200
212
petroleum_exporting	0.051699 sec.
224
stalled	0.049945 sec.
al_libi	0.052388 sec.
206
bands_like	0.039550 sec.
commodore_business	0.051874 sec.
201
201
ursa_minor	0.052306 sec.
205
astronomical_society	0.043762 sec.
201
200
214
202
colin_powell	0.051239 sec.
214
gaul	0.053030 sec.
national_academy	0.037972 sec.
200
203
gun_barrel	0.031516 sec.
cheated	0.050845 sec.
concentration_camps	0.031891 sec.
giant	0.037769 sec.
many_cultures	0.028380 sec.
203
definitions	0.033275 sec.
202
sir_winston	0.053370 sec.
slide_rule	0.053253 sec.
200
204
br_nsted	0.050174 sec.
experiencing	0.038706 sec.
222
palomar	0.040889 sec.
205
202
200
prejudicial	0.031480 sec.
205
208
204
no_reliable	0.052096 sec.
phase_ii	0.027709 sec.
five_ht	0.027970 sec.
208
haig	0.032893 sec.
melted	0.050756 sec.
208
216
201
dowry	0.047838 sec.
200
210
too_little	0.053393 sec.
201
exploits	0.051658 sec.
declines	0.051905 sec.
sergey	0.038751 sec.
subtitles	0.0499

203
203
204
francois	0.047385 sec.
204
202
lux	0.042483 sec.
inhalation	0.053160 sec.
stressed_syllable	0.038838 sec.
207
latter_day	0.031829 sec.
208
corn_syrup	0.035472 sec.
mongol_invasion	0.028335 sec.
cdot_left	0.040476 sec.
203
204
gibb	0.043087 sec.
205
maxillofacial	0.046872 sec.
204
optative	0.044622 sec.
201
203
precise_dates	0.051175 sec.
202
equator	0.044640 sec.
freedman	0.050392 sec.
209
205
license	0.032696 sec.
suspensions	0.050576 sec.
whispered	0.052004 sec.
200
205
203
bombardier	0.041082 sec.
atlantans	0.035031 sec.
205
207
201
204
opportunity_cost	0.053609 sec.
201
217
dilute	0.050164 sec.
222
physiologist	0.034951 sec.
hastened	0.043360 sec.
web_portal	0.049270 sec.
relentlessly	0.035609 sec.
sniff	0.034163 sec.
rebellious	0.033533 sec.
200
202
200
illegal_activities	0.031490 sec.
201
201
musical_theatre	0.048745 sec.
sms	0.043270 sec.
200
mockery	0.050244 sec.
resettlement	0.034801 sec.
204
algol	0.033392 sec.
200
pay_tribute	0.055078 sec.
airline_industry	0.0355

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.


In [79]:
import Pyro4

uri = "PYRO:obj_cca6f5c497464fafa29202c219ceda01@localhost:34932"

G = Pyro4.Proxy(uri)         
G.get_neighbors("man")

{'_atlas': {'a': {'weight': 0.354964},
  'about': {'weight': 0.426453},
  'across': {'weight': 0.376399},
  'action_potential': {'weight': 0.318236},
  'advanced_micro': {'weight': 0.2885},
  'affricates': {'weight': 0.314956},
  'africanus': {'weight': 0.33221},
  'aide': {'weight': 0.327552},
  'albedo': {'weight': 0.400208},
  'altair': {'weight': 0.29995},
  'an': {'weight': 0.299977},
  'analog_signals': {'weight': 0.324974},
  'anarchism': {'weight': 0.299189},
  'anarchist': {'weight': 0.344522},
  'anarchists': {'weight': 0.362887},
  'and': {'weight': 0.373304},
  'andrew_dickson': {'weight': 0.311741},
  'animaniacs': {'weight': 0.297462},
  'another': {'weight': 0.287766},
  'anti_virus': {'weight': 0.350922},
  'apocalypse_now': {'weight': 0.342072},
  'apollo': {'weight': 0.335858},
  'archery': {'weight': 0.282096},
  'are': {'weight': 0.41309},
  'areas': {'weight': 0.312454},
  'arithmetic_mean': {'weight': 0.322616},
  'as': {'weight': 0.344255},
  'asperger_s': {'weig